# Lab 12 - Sessionalization


## Advait Deochakke - 20BCE1143

In [23]:
import pandas as pd
from datetime import datetime, timedelta

# Load log data into a Pandas DataFrame
# log_data = pd.read_csv('log_file.csv')
log_data = pd.read_csv('given_log.csv') # read the file

# Convert timestamps to datetime objects
# log_data['Time'] = pd.to_datetime(log_data['Time'], format='%Y-%m-%d %H:%M:%S')
log_data['Time'] = pd.to_datetime(log_data['Time'], format='%H:%M') # time stuff

# H1: Total session duration may not exceed a threshold Theta.
# Given t0, the timestamp for the first request in a constructed session S,
# the request with a timestamp t is assigned to S, if (t - t0) <= Theta.
# H3 is included - Time&Agent matching
def assign_sessions_h1(log_data, theta):
    sessions = []
    current_session = []
    last_request_time = None
    cur_user = None
    for index, row in log_data.iterrows():
        if last_request_time is None or (row['Time'] - last_request_time) > theta or (row['IP'], row['Agent']) != cur_user:
            current_session = []
            sessions.append(current_session)
        current_session.append(index)
        last_request_time = row['Time']
        cur_user = (row['IP'], row['Agent'])
    return sessions

# H2: Total time spent on a page may not exceed a threshold delta.
# Given t1, the timestamp for the request assigned to constructed session S,
# the next request with timestamp t2 is assigned to S, if (t2 - t1) <= delta.
# H3 is included - Time&Agent matching
def assign_sessions_h2(log_data, delta):
    sessions = []
    current_session = []
    last_request_time = None
    cur_user = None
    last_request_url = None #referer
    for index, row in log_data.iterrows():
        if last_request_time is None or (row['Time'] - last_request_time) > delta or (row['IP'], row['Agent']) != cur_user or row['Ref'] != last_request_url:#row['Referer URL'] != last_request_url:
            # if last_request_time is None:
            #     print("last request none")
            # elif row['URL visited'] != last_request_url:
            #     print("row['URL visited'] != last_request_url:")
            # else:
            #     print("(row['Time'] - last_request_time) > delta")
            current_session = []
            sessions.append(current_session)
        current_session.append(index)
        last_request_time = row['Time']
        cur_user = (row['IP'], row['Agent'])
        last_request_url = row['URL']#row['URL visited']
    return sessions

# heuristic 1 and 2 already include the 3rd heuristic - IP&Agent matching in them.
sessions_h1 = assign_sessions_h1(log_data, timedelta(minutes=30))
sessions_h2 = assign_sessions_h2(log_data, timedelta(minutes=10))

# Print the sessions
for session in sessions_h1:
    print('H1 session:', session)

for session in sessions_h2:
    print('H2 session:', session)


H1 session: [0, 1]
H1 session: [2, 3, 4]
H1 session: [5]
H1 session: [6]
H1 session: [7]
H1 session: [8]
H1 session: [9, 10, 11, 12]
H1 session: [13, 14]
H1 session: [15]
H1 session: [16, 17, 18]
H2 session: [0, 1]
H2 session: [2, 3]
H2 session: [4]
H2 session: [5]
H2 session: [6]
H2 session: [7]
H2 session: [8]
H2 session: [9, 10]
H2 session: [11]
H2 session: [12]
H2 session: [13, 14]
H2 session: [15]
H2 session: [16]
H2 session: [17, 18]


In [24]:
print(log_data)
# used OCR to turn it into a .csv file

                  Time       IP URL  Ref          Agent
0  1900-01-01 00:01:00  1.2.3.4   A  NaN      IE5;Win2k
1  1900-01-01 00:09:00  1.2.3.4   B    A      IE5;Win2k
2  1900-01-01 00:10:00  2.3.4.5   C  NaN  IE6;WinXP;SP1
3  1900-01-01 00:12:00  2.3.4.5   B    C  IE6;WinXP;SP1
4  1900-01-01 00:15:00  2.3.4.5   E    C  IE6;WinXP;SP1
5  1900-01-01 00:19:00  1.2.3.4   C    A      IE5;Win2k
6  1900-01-01 00:22:00  2.3.4.5   D    B  IE6;WinXP;SP1
7  1900-01-01 00:22:00  1.2.3.4   A  NaN  IE6;WinXP;SP2
8  1900-01-01 00:25:00  1.2.3.4   E    C      IE5;Win2k
9  1900-01-01 00:25:00  1.2.3.4   C    A  IE6;WinXP;SP2
10 1900-01-01 00:33:00  1.2.3.4   B    C  IE6;WinXP;SP2
11 1900-01-01 00:58:00  1.2.3.4   D    B  IE6;WinXP;SP2
12 1900-01-01 01:10:00  1.2.3.4   E    D  IE6;WinXP;SP2
13 1900-01-01 01:15:00  1.2.3.4   A  NaN      IE5;Win2k
14 1900-01-01 01:16:00  1.2.3.4   C    A      IE5;Win2k
15 1900-01-01 01:17:00  1.2.3.4   F    C  IE6;WinXP;SP2
16 1900-01-01 01:26:00  1.2.3.4   F    C      IE